<a href="https://colab.research.google.com/github/DHarley22/Prediction_case_malaria_mozambique/blob/1-xgboost-predictor-python-version/model_building_mozambique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
%%capture
!pip install gdown


In [35]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
import os
import numpy as np
import xgboost as xgb
import sklearn
import gdown
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split as split_into_two
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor

In [36]:
%%capture

# The mozambique dataset is hosted here
# https://drive.google.com/file/d/1uHGOjxlxfDYY5E-aL9HVRGvheDa_wNyq/view?usp=sharing
dataset_drive_id = '1uHGOjxlxfDYY5E-aL9HVRGvheDa_wNyq'
dataset_output_file = 'downloaded_sheet.xlsx'

# The shape file relatd to the dataset is hosted here
# https://drive.google.com/drive/folders/14UJ7ZXWmNeL28sYAv6dObNsC42kQr4Ja?usp=sharing
shape_file_drive_id = '14UJ7ZXWmNeL28sYAv6dObNsC42kQr4Ja'
shape_output_folder = 'shape_files'

gdown.download_folder(id=shape_file_drive_id, output=shape_output_folder, quiet=False)
gdown.download(id=dataset_drive_id, output=dataset_output_file, quiet=False)


In [37]:
dataset = pd.read_csv(f"/content/{dataset_output_file}")
shape_file = gpd.read_file(f"/content/{shape_output_folder}/shape_file.shp")

In [38]:
# (Left) join the dataset to the shape_file using the "district" field
shaped_dataset = dataset.merge(
    shape_file[["district", "geometry"]],
    on="district",
    how="left"
)

# Convert the Dataframe into GeoDataframe and add some fields to it
shaped_dataset =  gpd.GeoDataFrame(shaped_dataset, geometry="geometry")

shaped_dataset["area"] = shaped_dataset.geometry.area
shaped_dataset["centroid_x"] = shaped_dataset.geometry.centroid.x
shaped_dataset["centroid_y"] = shaped_dataset.geometry.centroid.y

<ipython-input-38-6078cb333f54>:11: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  shaped_dataset["area"] = shaped_dataset.geometry.area
<ipython-input-38-6078cb333f54>:12: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  shaped_dataset["centroid_x"] = shaped_dataset.geometry.centroid.x
<ipython-input-38-6078cb333f54>:13: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  shaped_dataset["centroid_y"] = shaped_dataset.geometry.centroid.y


### Missing data handling

Rules:

- If {some condition} then {we do this}
- If {some condition} then {we do this}

In [39]:
# Handle missing data
# TODO:

### Test/Val/Test Split

Rules:

- We will use


In [43]:
TARGET_DISEASE_FIELD = "malaria_cases_u5"


# Relevant features
relevant_features = [
    # Environemental features
    'tmin', 'tmax', 'precipitation', 'ndvi','RH',
    # Socio-economic features
    'prop_poor', 'prop_Rural', 'prop_drinking_TreatedWater',
    # Spatial features
     'centroid_x', 'centroid_y'
]

# Test dataset predicate; use lines between March and June for testing
test_data_predicate = (shaped_dataset['year'] == 2018) & (shaped_dataset['month'].between(3, 6))

# Test dataset
X_test = shaped_dataset[test_data_predicate][relevant_features]
y_test = shaped_dataset[test_data_predicate][TARGET_DISEASE_FIELD]

# The rest of the dataset separated as X, y
X = shaped_dataset[~test_data_predicate][relevant_features]
y = shaped_dataset[~test_data_predicate][TARGET_DISEASE_FIELD]

print(X.shape, y.shape)
X_train, X_val, y_train, y_val = split_into_two(X, y, test_size=0.3, random_state=42)




(5088, 10) (5088,)


In [46]:
!pip install --upgrade xgboost scikit-learn

In [52]:
import xgboost as xgb

# Convert data to DMatrix
dtrain = xgb.DMatrix(X_train, label=y_train)

# Set parameter grid
param_grid = [
    {'max_depth': 3, 'learning_rate': 0.1, 'n_estimators': 100},
    {'max_depth': 5, 'learning_rate': 0.01, 'n_estimators': 500},
    # Add more combinations as needed
]

# Evaluate models
best_params = None
best_rmse = float('inf')
for params in param_grid:
    cv_results = xgb.cv(
        params=params,
        dtrain=dtrain,
        num_boost_round=500,
        nfold=5,
        metrics='rmse',
        early_stopping_rounds=50
    )
    mean_rmse = cv_results['test-rmse-mean'].min()
    if mean_rmse < best_rmse:
        best_rmse = mean_rmse
        best_params = params

print("Best Parameters:", best_params)
print("Best RMSE:", best_rmse)


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [01:15:51] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [01:15:55] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


Best Parameters: {'max_depth': 3, 'learning_rate': 0.1, 'n_estimators': 100}
Best RMSE: 879.7314672992077


In [ ]:
# xgb_cv_malaria = xgb.cv(
#     params=params,
#     dtrain=dtrain_malaria,
#     nfold=5,                      # 5-fold cross-validation
#     num_boost_round=5000,         # Maximum boosting rounds
#     early_stopping_rounds=150,    # Stop if no improvement for 150 rounds
#     metrics='rmse',               # Use RMSE for evaluation
#     as_pandas=True,               # Return results as a DataFrame
#     seed=0,
#     verbose_eval=50               # Print evaluation every 50 rounds
# )

# # Optimal number of boosting rounds
# best_boost_rounds = len(xgb_cv_malaria)
# print(f"Optimal Number of Rounds: {best_boost_rounds}")

# # Train final model with early stopping results
# bst_malaria = xgb.train(
#     params,
#     dtrain_malaria,
#     num_boost_round=best_boost_rounds,
#     evals=[(dtrain_malaria, 'train'), (dval_malaria, 'val')],
#     verbose_eval=50
# )
